# PROJET SEATTLE ENERGY BENCHMARKING
## Notebook 00 : initialisation de l'environnement et ingestion des données

---

### Identité du document
* **Statut :** Phase 1 (exploration & prototypage)
* **Date de création :** 26 Décembre 2025
* **Dernière mise à jour :** 29 Décembre 2025
* **Dépendances notebooks**: Aucune

### Description
Ce notebook établit les fondations du projet. Il assure la mise en place de l'infrastructure et la configuration des outils de gestion . L'objectif est de garantir une base reproductible avant d'entamer l'audit de qualité.

### Objectifs principaux
1. Initialiser le système de gestion de configuration via Hydra.
2. Déployer l'arborescence des répertoires du projet.
3. Récupérer et charger le jeu de données source (Immuable).
4. Établir un premier diagnostic structurel des données chargées.

### Dépendances critiques
* `hydra` : Gestion de la configuration.
* `src.utils` & `src.data` : Modules internes de support.

### LIVRABLES
1. Environnement technique initialisé (Hydra, Dossiers).
2. Dataset brut dans `data/raw/`.
3. Dictionnaire de données initial (`reports/data_dictionary.md`).
4. Note de synthèse structurelle.
5. Notebook propre

---

# Table des Matières
- [Section 0 : Importation des packages ](#section-1--configuration)
- [Section 1 : Configuration et initialisation ](#section-1--configuration)
- [Section 2 : Contextualisation & provenance](#section-2--contextualisation--provenance)
- [Section 3 : Chargement et inspection structurelle](#section-3--ingestion-et-premier-contact)
- [Section 4 : Typologie des variables](#section-4--typologie-et-cartographie-des-variables)
- [Section 5 : Focus sur la variable cible](#section-5--analyse-de-la-variable-cible)
- [Section 6 : Identification des redondances apparentes](#section-6--étude-des-redondances-et-unités)
- [Section 7 :Premières incohérences détectées](#section-7--détection-des-signaux-dalerte-immédiats)
- [Section 8 : Synthèse et génération du dictionnaire](#section-8--synthèse-et-génération-du-dictionnaire)
---

# Section 0 : Importation des packages

In [1]:
import logging
import pandas as pd
from pathlib import Path


# Import des fonctions utilitaires 

import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
SRC_PATH = PROJECT_ROOT / "src"

if str(SRC_PATH) not in sys.path:
    sys.path.insert(0, str(SRC_PATH))


from data.load_data import load_data_raw
from utils.config_loader import load_config, create_directories
from utils.eda_logger import setup_eda_logger
 


# Section 1:Configuration et initialisation

In [2]:
# Chargement de la configuration principale (Hydra)
cfg = load_config()

# Initialisation du logger
setup_eda_logger(cfg)
logger = logging.getLogger(__name__)

# Création des dossiers nécessaires à l'exécution (raw, interim, processed, reports, etc.)
create_directories(cfg)


2025-12-31 00:16:22,374 - utils.config_loader - INFO - Répertoire prêt : C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2\data\raw
2025-12-31 00:16:22,374 - utils.config_loader - INFO - Répertoire prêt : C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2\data\interim
2025-12-31 00:16:22,374 - utils.config_loader - INFO - Répertoire prêt : C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2\data\processed
2025-12-31 00:16:22,374 - utils.config_loader - INFO - Répertoire prêt : C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2\figures
2025-12-31 00:16:22,374 - utils.config_loader - INFO - Répertoire prêt : C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2\reports


# Section 2 : Chargement et inspection structurelle

Cette section se concentre sur le chargement initial des données à partir du fichier CSV et une inspection structurelle de base. L'objectif est d'obtenir une première vue d'ensemble du dataset, de vérifier son intégrité et d'identifier tout problème évident au niveau du format ou de la structure.

## Importation du CSV

In [3]:
# Chargement des données brutes
df_raw = load_data_raw(cfg)

# Validation dimensions
n_rows, n_cols = df_raw.shape
logger.info(f"Dataset chargé ({n_rows} lignes, {n_cols} colonnes)")



2025-12-31 00:16:22,429 - data.load_data - INFO - DataFrame chargé : 3376 lignes, 46 colonnes
2025-12-31 00:16:22,448 - data.load_data - INFO -  Intégrité des données validée (Aucune modification détectée).
2025-12-31 00:16:22,448 - __main__ - INFO - Dataset chargé (3376 lignes, 46 colonnes)


---
Le chargement nous donne un DataFrame avec 3376 lignes et 46 colonnes.

## Affichage pour première impression visuelle

In [4]:
# Forcer pandas à afficher toutes les colonnes
pd.set_option("display.max_columns", None)

# Forcer pandas à afficher toutes les lignes (si besoin)
pd.set_option("display.max_rows", None)

# Forcer pandas à afficher toute la largeur (évite les "...")
pd.set_option("display.width", None)



In [5]:
# Affichage des premières lignes
print("Premières 5 lignes :")
display(df_raw.head(5))

# Affichage des dernières lignes
print("Dernières 5 lignes :")
display(df_raw.tail(5))

# Affichage d'un échantillon aléatoire
print("Échantillon aléatoire de 20 lignes :")
display(df_raw.sample(20))

Premières 5 lignes :


,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,TaxParcelIdentificationNumber,CouncilDistrictCode,Neighborhood,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),ListOfAllPropertyUseTypes,LargestPropertyUseType,LargestPropertyUseTypeGFA,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,YearsENERGYSTARCertified,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,Comments,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity
0,1,2016,NonResidential,Hotel,Mayflower park hotel,405 Olive way,Seattle,WA,98101.0,0659000030,7,DOWNTOWN,47.61220,-122.33799,1927,1.0,12,88434,0,88434,Hotel,Hotel,88434.0,NaN,NaN,NaN,NaN,NaN,60.0,81.699997,84.300003,182.500000,189.000000,7226362.5,7456910.0,2003882.00,1.156514e+06,3946027.0,12764.52930,1276453.0,False,NaN,Compliant,NaN,249.98,2.83
1,2,2016,NonResidential,Hotel,Paramount Hotel,724 Pine street,Seattle,WA,98101.0,0659000220,7,DOWNTOWN,47.61317,-122.33393,1996,1.0,11,103566,15064,88502,"Hotel, Parking, Restaurant",Hotel,83880.0,Parking,15064.0,Restaurant,4622.0,NaN,61.0,94.800003,97.900002,176.100006,179.399994,8387933.0,8664479.0,0.00,9.504252e+05,3242851.0,51450.81641,5145082.0,False,NaN,Compliant,NaN,295.86,2.86
2,3,2016,NonResidential,Hotel,5673-The Westin Seattle,1900 5th Avenue,Seattle,WA,98101.0,0659000475,7,DOWNTOWN,47.61393,-122.33810,1969,1.0,41,956110,196718,759392,Hotel,Hotel,756493.0,NaN,NaN,NaN,NaN,NaN,43.0,96.000000,97.699997,241.899994,244.100006,72587024.0,73937112.0,21566554.00,1.451544e+07,49526664.0,14938.00000,1493800.0,False,NaN,Compliant,NaN,2089.28,2.19
3,5,2016,NonResidential,Hotel,HOTEL MAX,620 STEWART ST,Seattle,WA,98101.0,0659000640,7,DOWNTOWN,47.61412,-122.33664,1926,1.0,10,61320,0,61320,Hotel,Hotel,61320.0,NaN,NaN,NaN,NaN,NaN,56.0,110.800003,113.300003,216.199997,224.000000,6794584.0,6946800.5,2214446.25,8.115253e+05,2768924.0,18112.13086,1811213.0,False,NaN,Compliant,NaN,286.43,4.67
4,8,2016,NonResidential,Hotel,WARWICK SEATTLE HOTEL (ID8),401 LENORA ST,Seattle,WA,98121.0,0659000970,7,DOWNTOWN,47.61375,-122.34047,1980,1.0,18,175580,62000,113580,"Hotel, Parking, Swimming Pool",Hotel,123445.0,Parking,68009.0,Swimming Pool,0.0,NaN,75.0,114.800003,118.699997,211.399994,215.600006,14172606.0,14656503.0,0.00,1.573449e+06,5368607.0,88039.98438,8803998.0,False,NaN,Compliant,NaN,505.01,2.88


Dernières 5 lignes :


,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,TaxParcelIdentificationNumber,CouncilDistrictCode,Neighborhood,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),ListOfAllPropertyUseTypes,LargestPropertyUseType,LargestPropertyUseTypeGFA,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,YearsENERGYSTARCertified,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,Comments,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity
3371,50222,2016,Nonresidential COS,Office,Horticulture building,1600 S Dakota St,Seattle,WA,NaN,1624049080,2,GREATER DUWAMISH,47.56722,-122.31154,1990,1.0,1,12294,0,12294,Office,Office,12294.0,NaN,NaN,NaN,NaN,NaN,46.0,69.099998,76.699997,161.699997,176.100006,8.497457e+05,9.430032e+05,0.0,153655.0000,5.242709e+05,3254.750244,3.254750e+05,True,NaN,Error - Correct Default Data,NaN,20.94,1.70
3372,50223,2016,Nonresidential COS,Other,International district/Chinatown CC,719 8th Ave S,Seattle,WA,NaN,3558300000,2,DOWNTOWN,47.59625,-122.32283,2004,1.0,1,16000,0,16000,Other - Recreation,Other - Recreation,16000.0,NaN,NaN,NaN,NaN,NaN,NaN,59.400002,65.900002,114.199997,118.900002,9.502762e+05,1.053706e+06,0.0,116221.0000,3.965461e+05,5537.299805,5.537300e+05,False,NaN,Compliant,NaN,32.17,2.01
3373,50224,2016,Nonresidential COS,Other,Queen Anne Pool,1920 1st Ave W,Seattle,WA,NaN,1794501150,7,MAGNOLIA / QUEEN ANNE,47.63644,-122.35784,1974,1.0,1,13157,0,13157,"Fitness Center/Health Club/Gym, Other - Recrea...",Other - Recreation,7583.0,Fitness Center/Health Club/Gym,5574.0,Swimming Pool,0.0,NaN,NaN,438.200012,460.100006,744.799988,767.799988,5.765898e+06,6.053764e+06,0.0,525251.6875,1.792159e+06,39737.390630,3.973739e+06,False,NaN,Compliant,NaN,223.54,16.99
3374,50225,2016,Nonresidential COS,Mixed Use Property,South Park Community Center,8319 8th Ave S,Seattle,WA,NaN,7883603155,1,GREATER DUWAMISH,47.52832,-122.32431,1989,1.0,1,14101,0,14101,"Fitness Center/Health Club/Gym, Food Service, ...",Other - Recreation,6601.0,Fitness Center/Health Club/Gym,6501.0,Pre-school/Daycare,484.0,NaN,NaN,51.000000,55.500000,105.300003,110.800003,7.194712e+05,7.828413e+05,0.0,102248.0000,3.488702e+05,3706.010010,3.706010e+05,False,NaN,Compliant,NaN,22.11,1.57
3375,50226,2016,Nonresidential COS,Mixed Use Property,Van Asselt Community Center,2820 S Myrtle St,Seattle,WA,NaN,7857002030,2,GREATER DUWAMISH,47.53939,-122.29536,1938,1.0,1,18258,0,18258,"Fitness Center/Health Club/Gym, Food Service, ...",Other - Recreation,8271.0,Fitness Center/Health Club/Gym,8000.0,Pre-school/Daycare,1108.0,NaN,NaN,63.099998,70.900002,115.800003,123.900002,1.152896e+06,1.293722e+06,0.0,126774.3984,4.325542e+05,7203.419922,7.203420e+05,False,NaN,Compliant,NaN,41.27,2.26


Échantillon aléatoire de 20 lignes :


,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,TaxParcelIdentificationNumber,CouncilDistrictCode,Neighborhood,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),ListOfAllPropertyUseTypes,LargestPropertyUseType,LargestPropertyUseTypeGFA,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,YearsENERGYSTARCertified,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,Comments,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity
2694,26653,2016,Multifamily LR (1-4),Low-Rise Multifamily,University Park Apartments,5240 15th Ave NE,Seattle,WA,98105.0,8823900835,4,NORTHEAST,47.66781,-122.31161,1981,1.0,4,28753,5782,22971,"Multifamily Housing, Parking",Multifamily Housing,22971.0,Parking,5782.0,NaN,NaN,NaN,57.0,31.900000,34.400002,100.199997,107.900002,7.332785e+05,7.896673e+05,0.00,2.149116e+05,733278.0,0.000000,0.0,True,NaN,Error - Correct Default Data,NaN,5.11,0.18
2407,25546,2016,NonResidential,Mixed Use Property,Compass Housing Alliance - Pioneer Square,210 Alaskan Way S / 77 S. Washington St,Seattle,WA,98104.0,5247800080,7,DOWNTOWN,47.60050,-122.33514,2005,0.0,5,57600,0,57600,"Financial Office, Multifamily Housing, Parking...",Residence Hall/Dormitory,27600.0,Multifamily Housing,16000.0,Financial Office,7000.0,NaN,NaN,138.000000,145.399994,277.000000,286.899994,6.983736e+06,7.355579e+06,0.00,9.372571e+05,3197921.0,37858.144530,3785814.0,False,NaN,Compliant,NaN,223.36,3.88
1154,21115,2016,NonResidential,Small- and Mid-Sized Office,Administration - Cassel Crag,1218 Terry Ave,Seattle,WA,98101.0,1978200285,3,EAST,47.61098,-122.32766,1925,1.0,4,39760,0,39760,Office,Office,39760.0,NaN,NaN,NaN,NaN,NaN,92.0,62.299999,69.000000,99.099998,107.000000,2.478960e+06,2.741556e+06,1739620.75,1.501247e+05,512225.0,2271.139893,227114.0,False,NaN,Compliant,NaN,149.91,3.77
2130,24556,2016,Multifamily MR (5-9),Mid-Rise Multifamily,Pontedera Condominiums,827 Hiawatha Place South,Seattle,WA,98144.0,6843650000,3,CENTRAL,47.59515,-122.30985,2009,0.0,6,105903,0,105903,"Multifamily Housing, Office",Multifamily Housing,96340.0,Office,9563.0,NaN,NaN,NaN,92.0,34.400002,35.700001,78.400002,80.699997,3.644085e+06,3.784060e+06,0.00,6.279038e+05,2142408.0,15016.769530,1501677.0,False,NaN,Compliant,NaN,94.69,0.89
3315,50029,2016,Multifamily HR (10+),High-Rise Multifamily,Walton Lofts,75 Vine St,Seattle,WA,98121.0,0653000195,7,DOWNTOWN,47.61428,-122.35056,2014,1.0,12,123686,0,123686,"Multifamily Housing, Parking",Multifamily Housing,152301.0,Parking,0.0,NaN,NaN,NaN,77.0,29.700001,30.000000,76.800003,77.800003,4.524791e+06,4.573978e+06,0.00,9.739330e+05,3323059.0,12017.311520,1201731.0,False,NaN,Compliant,NaN,86.99,0.70
2805,27020,2016,NonResidential,Residence Hall,Alpha Delta Pi Chapter House,1805 NE 47th,Seattle,WA,98177.0,8823902825,4,NORTHEAST,47.66281,-122.30819,1950,1.0,3,29084,0,29084,Residence Hall/Dormitory,Residence Hall/Dormitory,26000.0,NaN,NaN,NaN,NaN,NaN,66.0,62.799999,67.000000,111.000000,116.500000,1.632499e+06,1.741865e+06,0.00,1.643214e+05,560665.0,10718.340820,1071834.0,False,NaN,Compliant,NaN,60.83,2.09
1837,23560,2016,Multifamily LR (1-4),Low-Rise Multifamily,Tramonti at Lake Union,2920 Eastlake Ave. East,Seattle,WA,98102.0,8669900000,4,LAKE UNION,47.64786,-122.32378,2004,1.0,4,62504,11128,51376,Multifamily Housing,Multifamily Housing,53931.0,NaN,NaN,NaN,NaN,NaN,50.0,18.700001,18.700001,58.599998,58.599998,1.006990e+06,1.006990e+06,0.00,2.948422e+05,1006002.0,9.880000,988.0,False,NaN,Compliant,NaN,7.07,0.11
3337,50062,2016,NonResidential,Hotel,Olympic Hospitality,3926 Aurora Ave N,Seattle,WA,98103.0,1931300480,4,LAKE UNION,47.65476,-122.34732,2014,1.0,4,126823,41539,85284,"H

---
La colonne 'DataYear' est constante, ce qui est attendu puisque le dataset porte sur une seule campagne annuelle. Cette variable n’apporte donc pas de variance et pourra être ignorée dans la modélisation.

On observe une présence importante de valeurs manquantes dans les colonnes liées aux usages secondaires et tertiaires des bâtiments (SecondLargestPropertyUseType, ThirdLargestPropertyUseType, YearsENERGYSTARCertified, Comments). Cela traduit soit une absence d’information, soit des cas où le bâtiment n’a qu’un usage principal. Ces colonnes nécessiteront un exament approfondi.

Les colonnes lié à l'energie  montrent de nombreux zéros :  
- *PropertyGFAParking* est souvent à 0, ce qui est logique pour les bâtiments sans parking.  
- *SteamUse(kBtu)* et *NaturalGas(kBtu)* présentent des 0.00, ce qui peut indiquer une absence réelle de consommation ou une donnée manquante codée comme zéro. 
> Il faudra distinguer les deux cas pour éviter de biaiser les modèles. 


Electricity(kWh),la quantité annuelle d'éléctricité consommée présente d'énormes differences (ordre de grandeur),selon les batiments.

La colonne **ComplianceStatus** ( **Si une propriété a satisfait aux exigences d'analyse comparative énergétique pour l'année de déclaration en cours**), est presque toujours “Compliant”, ce qui suggère que la majorité des bâtiments respectent la réglementation. 

La colonne **Outlier** est vide avec des NaN, ce qui peut indiquer que les anomalies sont rares ou non encore identifiées.


## Informations structurelles

In [10]:
df_raw.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3376 entries, 0 to 3375
Data columns (total 46 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   OSEBuildingID                    3376 non-null   int64  
 1   DataYear                         3376 non-null   int64  
 2   BuildingType                     3376 non-null   object 
 3   PrimaryPropertyType              3376 non-null   object 
 4   PropertyName                     3376 non-null   object 
 5   Address                          3376 non-null   object 
 6   City                             3376 non-null   object 
 7   State                            3376 non-null   object 
 8   ZipCode                          3360 non-null   float64
 9   TaxParcelIdentificationNumber    3376 non-null   object 
 10  CouncilDistrictCode              3376 non-null   int64  
 11  Neighborhood                     3376 non-null   object 
 12  Latitude            

In [9]:
# Liste des colonnes
print("Colonnes :")
print(df_raw.columns.tolist())

Colonnes :
['OSEBuildingID', 'DataYear', 'BuildingType', 'PrimaryPropertyType', 'PropertyName', 'Address', 'City', 'State', 'ZipCode', 'TaxParcelIdentificationNumber', 'CouncilDistrictCode', 'Neighborhood', 'Latitude', 'Longitude', 'YearBuilt', 'NumberofBuildings', 'NumberofFloors', 'PropertyGFATotal', 'PropertyGFAParking', 'PropertyGFABuilding(s)', 'ListOfAllPropertyUseTypes', 'LargestPropertyUseType', 'LargestPropertyUseTypeGFA', 'SecondLargestPropertyUseType', 'SecondLargestPropertyUseTypeGFA', 'ThirdLargestPropertyUseType', 'ThirdLargestPropertyUseTypeGFA', 'YearsENERGYSTARCertified', 'ENERGYSTARScore', 'SiteEUI(kBtu/sf)', 'SiteEUIWN(kBtu/sf)', 'SourceEUI(kBtu/sf)', 'SourceEUIWN(kBtu/sf)', 'SiteEnergyUse(kBtu)', 'SiteEnergyUseWN(kBtu)', 'SteamUse(kBtu)', 'Electricity(kWh)', 'Electricity(kBtu)', 'NaturalGas(therms)', 'NaturalGas(kBtu)', 'DefaultData', 'Comments', 'ComplianceStatus', 'Outlier', 'TotalGHGEmissions', 'GHGEmissionsIntensity']


Voir le dictionnaire des donnée dans raports/ pour des définitions

On constate que les types des variables sont coherent.(un int déguisé en float ne pose pas de probléme)

>La variable zipcode est identifié comme int mais dois être traité comme une variable qualitatif

> colonnes :46 au total, couvrant identifiants, localisation, caractéristiques, usages, performances, consommations, émissions

>le 'WN' signifie Weather Normalized, une normalisation importante pour comparer les années

Nous avons aussi quelques variables atypiques :

> **DefaultData** booléen pour usage de données par défaut
>**Outlier** Si une propriété est une valeur aberrante élevée ou faible (O/N)

Cela necessite une documentation plus approfondi

# Section 3 : Typologie des variables

Dans cette section, nous présentons le dictionnaire des données. Afin de garantir la cohérence et la fidélité aux sources, nous avons repris la documentation officielle et en avons traduit automatiquement les éléments essentiels. Les informations pertinentes ont été extraites et adaptées pour ce projet, de manière à offrir une lecture directe et exploitable.

Le document complet est disponible dans le dossier  reports/notebooks/data_dictionary_auto.pdf. Il constitue la référence officielle pour la description des variables et sera utilisé comme support de documentation dans le cadre du projet.

## Structure et cohérence des variables

On remarque que le dictionnaire est très riche et couvre à la fois des identifiants administratifs (OSEBuildingID, TaxParcelIdentificationNumber), des informations géographiques (adresse, latitude, longitude, quartier), et des caractéristiques physiques des bâtiments (année de construction, nombre d’étages, surface totale). Cette structuration permet de relier chaque enregistrement à une entité réelle et de contextualiser les mesures énergétiques.
Un point intéressant est la présence de variables redondantes ou complémentaires, comme PropertyGFATotal, PropertyGFAParking et PropertyGFABuilding(s).

## Données énergétiques et enjeux métier


SiteEUI, SourceEUI, consommation d’électricité, gaz, vapeur, émissions de GES, Ces variables traduisent directement l’objectif du benchmarking : comparer les bâtiments entre eux et identifier les leviers de réduction.
On note aussi la présence de champs qualitatifs comme ComplianceStatus ou Outlier, qui ajoutent une dimension réglementaire et statistique. Cela montre que le dataset n’est pas seulement descriptif, mais qu’il sert aussi à évaluer la conformité et détecter les anomalies.
Enfin, certaines colonnes sont très incomplètes (YearsENERGYSTARCertified, Comments, Outlier), ce qui rappelle une réalité humaine : la difficulté de collecter des données exhaustives. C’est une limite mais aussi une information précieuse sur la fiabilité et la représentativité des analyses.

## Classification des variables


## 1. Identification et localisation

Ces variables servent à identifier le bâtiment de manière unique et à le situer géographiquement.

| Variable | Type | Description |
| --- | --- | --- |
| **OSEBuildingID** | Numérique (ID) | Identifiant unique de la propriété. |
| **PropertyName** | Texte | Nom officiel du bâtiment. |
| **TaxParcelIdentificationNumber** | Texte | Identifiant fiscal de la parcelle (King County). |
| **Address** | Texte | Adresse physique. |
| **City** | Texte | Ville (Seattle). |
| **State** | Texte | État (WA). |
| **ZipCode** | Catégoriel | Code postal. |
| **CouncilDistrictCode** | Catégoriel | District municipal (1 à 7). |
| **Neighborhood** | Catégoriel | Zone de quartier définie par la ville. |
| **Latitude** | Numérique | Coordonnée géographique nord-sud. |
| **Longitude** | Numérique | Coordonnée géographique est-ouest. |

---

## 2. Caractéristiques morphologiques et temporelles

Variables décrivant la structure physique et l'âge du bâtiment.

| Variable | Type | Description |
| --- | --- | --- |
| **DataYear** | Temporel | Année de l'enregistrement (2016). |
| **YearBuilt** | Temporel | Année de construction ou de rénovation majeure. |
| **NumberofBuildings** | Numérique | Nombre de bâtiments sur le site. |
| **NumberofFloors** | Numérique | Nombre d'étages du bâtiment. |
| **PropertyGFATotal** | Numérique | Surface totale (Bâtiment + Parking) en pieds carrés. |
| **PropertyGFABuilding(s)** | Numérique | Surface brute hors parking. |
| **PropertyGFAParking** | Numérique | Surface totale des parkings. |

---

## 3. Typologie et usage des sols

Classification du bâtiment selon son activité principale et secondaire.

| Variable | Type | Description |
| --- | --- | --- |
| **BuildingType** | Catégoriel | Classification globale (ex: NonResidential, Campus). |
| **PrimaryPropertyType** | Catégoriel | Usage principal (>50%) selon Portfolio Manager. |
| **ListOfAllPropertyUseTypes** | Texte | Liste exhaustive de tous les usages sur le site. |
| **LargestPropertyUseType** | Catégoriel | Usage occupant la plus grande surface. |
| **LargestPropertyUseTypeGFA** | Numérique | Surface associée à l'usage le plus important. |
| **SecondLargestPropertyUseType** | Catégoriel | Deuxième usage par importance de surface. |
| **SecondLargestPropertyUseTypeGFA** | Numérique | Surface associée au deuxième usage. |
| **ThirdLargestPropertyUseType** | Catégoriel | Troisième usage par importance de surface. |
| **ThirdLargestPropertyUseTypeGFA** | Numérique | Surface associée au troisième usage. |

---

## 4. Performance et Certification

Indicateurs d'efficacité énergétique et labels environnementaux.

| Variable | Type | Description |
| --- | --- | --- |
| **ENERGYSTARScore** | Numérique (0-100) | Score de performance relative (EPA). |
| **YearsENERGYSTARCertified** | Texte | Historique des années de certification Energy Star. |

---

## 5. Intensité d'usage de l'Énergie (EUI)

Ratios de consommation par unité de surface, normalisés ou non (Indicateurs clés de performance).

| Variable | Type | Unité | Description |
| --- | --- | --- | --- |
| **SiteEUI(kBtu/sf)** | Numérique | kBtu/sq.ft | Énergie consommée sur site par pied carré. |
| **SiteEUIWN(kBtu/sf)** | Numérique | kBtu/sq.ft | EUI du site normalisé selon la météo. |
| **SourceEUI(kBtu/sf)** | Numérique | kBtu/sq.ft | Énergie à la source (inclut pertes réseau) par pied carré. |
| **SourceEUIWN(kBtu/sf)** | Numérique | kBtu/sq.ft | Source EUI normalisé selon la météo. |

---

## 6. Consommation Énergétique par source

Volumes de consommation bruts selon le type d'énergie.

| Variable | Type | Unité | Description |
| --- | --- | --- | --- |
| **SiteEnergyUse(kBtu)** | Numérique | kBtu | Consommation totale annuelle du site. |
| **SiteEnergyUseWN(kBtu)** | Numérique | kBtu | Consommation totale normalisée (météo). |
| **Electricity(kWh)** | Numérique | kWh | Consommation électrique brute. |
| **Electricity(kBtu)** | Numérique | kBtu | Consommation électrique convertie en kBtu. |
| **NaturalGas(therms)** | Numérique | therms | Consommation de gaz naturel brute. |
| **NaturalGas(kBtu)** | Numérique | kBtu | Consommation de gaz naturel convertie en kBtu. |
| **SteamUse(kBtu)** | Numérique | kBtu | Consommation de vapeur urbaine. |

---

## 7. Émissions de gaz à effet de serre 

Mesure de l'impact environnemental du bâtiment.

| Variable | Type | Unité | Description |
| --- | --- | --- | --- |
| **TotalGHGEmissions** | Numérique | Tonnes CO2e | Émissions totales de gaz à effet de serre. |
| **GHGEmissionsIntensity** | Numérique | kg CO2e/sq.ft | Émissions par unité de surface. |

---

## 8. qualité des données et conformité

Variables indiquant la fiabilité des informations et l'état réglementaire.

| Variable | Type | Description |
| --- | --- | --- |
| **DefaultData** | Booléen | Indique si des valeurs par défaut ont été utilisées. |
| **ComplianceStatus** | Catégoriel | Statut vis-à-vis de l'ordonnance de benchmarking. |
| **Outlier** | Catégoriel | Indique si la donnée est une valeur aberrante. |
| **Comments** | Texte | Remarques textuelles additionnelles (très peu rempli). |

---